In [1]:
#!pip install fairsearchcore

In [12]:
import fairsearchcore as fsc
import pandas as pd
from fairsearchcore.models import FairScoreDoc

In [13]:
data = pd.read_csv("data/tutors.csv")
dat_wp = data[data.webpage == "superprof"]
dat_wp.reset_index(inplace = True,drop=True)
dat_wp1 = data[data.webpage == "classgap"]
dat_wp1.reset_index(inplace = True,drop=True)
dat_wp2 = data[data.webpage == "tusclasesparticulares"]
dat_wp2.reset_index(inplace = True,drop=True)
dat_wp3 = data[data.webpage == "buscatuprofesor"]
dat_wp3.reset_index(inplace = True,drop=True)
dat_wp4 = data[data.webpage == "yoopie"]
dat_wp4.reset_index(inplace = True,drop=True)

In [14]:
print(data.groupby("genero").count().id_profesor[0]/(data.groupby("genero").count().id_profesor[0]+data.groupby("genero").count().id_profesor[1]))
print(dat_wp.groupby("genero").count().id_profesor[0]/(dat_wp.groupby("genero").count().id_profesor[0]+dat_wp.groupby("genero").count().id_profesor[1]))
print(dat_wp1.groupby("genero").count().id_profesor[0]/(dat_wp1.groupby("genero").count().id_profesor[0]+dat_wp1.groupby("genero").count().id_profesor[1]))
print(dat_wp2.groupby("genero").count().id_profesor[0]/(dat_wp2.groupby("genero").count().id_profesor[0]+dat_wp2.groupby("genero").count().id_profesor[1]))
print(dat_wp3.groupby("genero").count().id_profesor[0]/(dat_wp3.groupby("genero").count().id_profesor[0]+dat_wp3.groupby("genero").count().id_profesor[1]))
print(dat_wp4.groupby("genero").count().id_profesor[0]/(dat_wp4.groupby("genero").count().id_profesor[0]+dat_wp4.groupby("genero").count().id_profesor[1]))

0.5497507845671036
0.5012987012987012
0.49238578680203043
0.5897435897435898
0.6341463414634146
0.9029535864978903


In [15]:
data = pd.read_csv("data/tutors.csv")
dat_wp = data[data.materia == "Ciencias"]
dat_wp.reset_index(inplace = True,drop=True)
dat_wp1 = data[data.materia == "Deporte"]
dat_wp1.reset_index(inplace = True,drop=True)
dat_wp2 = data[data.materia == "Idiomas"]
dat_wp2.reset_index(inplace = True,drop=True)
dat_wp3 = data[data.materia == "Musica"]
dat_wp3.reset_index(inplace = True,drop=True)
dat_wp4 = data[data.materia == "Repaso"]
dat_wp4.reset_index(inplace = True,drop=True)

In [16]:
import numpy as np
np.unique(data.materia)

array(['Ciencias', 'Deporte', 'Idiomas', 'Musica', 'Repaso'], dtype=object)

In [17]:
def create_all_rankings(df):
    rankings = []
    rank = []
    for i,position in enumerate(df.posicion_top):
        if position == 1 and len(rank) == 0:
            rank.append(FairScoreDoc(position, None, df.genero[i] == "female"))
        elif position == 1 and len(rank) != 0:
            rankings.append([rank,df.webpage[i],df.materia[i]])
            rank = []
            rank.append(FairScoreDoc(position, None, df.genero[i] == "female"))
        else:
            rank.append(FairScoreDoc(position, None, df.genero[i] == "female"))
    return rankings

In [18]:
def max_p(ranking,alpha):
    res = 0
    if len(ranking) > 400:
        k = 400
    else:
        k = len(ranking)
    for p in np.arange(0.05,0.95,0.05):
        fair = fsc.Fair(k, p, alpha)
        if fair.is_fair(ranking[0:k]):
            res = p
    return res  

In [19]:
import numpy as np

In [20]:
c = create_all_rankings(data)

In [ ]:
from tqdm import tqdm
ps = []
webpage = []
materia = []
for rank in tqdm(c):
    if len(rank[0]) < 10:
        continue
    ps.append(max_p(rank[0],0.1))
    webpage.append(rank[1])
    materia.append(rank[2])

In [ ]:
dt = pd.DataFrame({"max_p":ps,"webpage":webpage,"materia":materia})
#dt.to_csv("max_p.csv")
#Materia
print(dt.groupby("materia").max_p.quantile(0.25))
print(dt.groupby("materia").max_p.quantile(0.5))
print(dt.groupby("materia").max_p.quantile(0.75))
print(dt.groupby("materia").max_p.mean())
#Webpage
print(dt.groupby("webpage").max_p.quantile(0.25))
print(dt.groupby("webpage").max_p.quantile(0.5))
print(dt.groupby("webpage").max_p.quantile(0.75))
print(dt.groupby("webpage").max_p.mean())
#Total
print(df.max_p.quantile(0.25))
print(df.max_p.quantile(0.5))
print(df.max_p.quantile(0.75))
print(df.max_p.mean())